# Задание 3. Титаник

In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from accuracy_dependence import normalize_data

## Чтение данных из файла

In [2]:
train_data = pd.read_csv('datasets/Titanic_train.csv', sep=',', header=0, index_col=0)
X_test_data = pd.read_csv('datasets/Titanic_X_test.csv', sep=',', header=0, index_col=0)
y_test_data = pd.read_csv('datasets/Titanic_y_test.csv', sep=',', header=0, index_col=0)

In [3]:
# Подготовка данных
train_data.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)
train_data.replace({'Embarked': {'C': 0, 'Q': 1, 'S': 2}}, inplace=True)
train_data.fillna(0, inplace=True)
# Подготовка неразмеченных данных
X_test_data.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)
X_test_data.replace({'Embarked': {'C': 0, 'Q': 1, 'S': 2}}, inplace=True)
X_test_data.fillna(0, inplace=True)

y_train_data = train_data['Survived']
X_train_data = train_data.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
X_test_data = X_test_data.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
X_train_data = normalize_data(X_train_data)
X_test_data = normalize_data(X_test_data)
# X_train, X_test, y_train, y_test = train_test_split(X_train_data, y_train_data, random_state=random.randint(0, 1000))

## Настройка сетки поиска параметров

In [11]:
gnb = GaussianNB()
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
search_grid = {'var_smoothing': np.logspace(0, -10, num=1000)}
gscv = GridSearchCV(gnb, param_grid=search_grid, cv=kf, n_jobs=-1, verbose=1)

In [12]:
gscv.fit(X_train_data, y_train_data)
# print(pd.DataFrame(gscv.cv_results_))
print(gscv.best_params_)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
{'var_smoothing': 0.9549485639791966}


In [13]:
print(gscv.score(X_test_data, y_test_data))

0.9019138755980861


In [14]:
labels = np.array(gscv.predict(X=X_test_data))
label0 = np.count_nonzero(labels == 0)
print("Утонувшие {}".format(str(label0)))
print("Выжившие {}".format(str(X_test_data.index.size - label0)))

Утонувшие 307
Выжившие 111
